<a href="https://colab.research.google.com/github/mengyaoo/FNL_GenesSelection/blob/main/07_TC1_Feature_Selection_For_Each_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the libraries

In [1]:
from __future__ import print_function

import os, sys, gzip, glob, json, time, argparse
import pandas as pd
import numpy as np
import keras
from pandas.io.json import json_normalize

from pandas.io.json import json_normalize
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from keras.utils import to_categorical
from keras import backend as K
from keras.layers import Input, Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten
from keras import optimizers
from keras.optimizers import SGD, Adam, RMSprop
from keras.models import Sequential, Model, model_from_json, model_from_yaml
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau

from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import sklearn.manifold as sk_manif
from keras.utils import to_categorical
from google.colab import drive
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
%matplotlib inline

# Data Preparation   

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read features and output files 
TC1data15 = pd.read_csv("/content/drive/My Drive/FNL_TC1/TC1-S1-data15-genename.tsv", sep="\t", low_memory = False)

In [ ]:
# dftm_FPKM_UQ = pd.read_csv('/content/drive/My Drive/FNL_TC1/dftm_FPKM_UQ.tsv', sep='\t', low_memory=False)

In [4]:
outcome = pd.read_csv('/content/drive/My Drive/FNL_TC1/TC1-outcome-data15-projectname.tsv', sep='\t')

In [5]:
def encode(data): 
    print('Shape of data (BEFORE encode): %s' % str(data.shape))
    encoded = to_categorical(data)
    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
    return encoded

In [ ]:
# outcome = encode(outcome['Project_id'])

In [6]:
TC1data15['project_id_name'] = outcome['Project_id_name']
TC1data15['project_id'] = outcome['Project_id']


In [7]:
TC1data15

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,NIPAL3,LAS1L,ENPP4,SEMA3F,CFTR,ANKIB1,CYP51A1,KRIT1,RAD52,MYH16,BAD,LAP3,CD99,HS3ST1,AOC1,WNT16,HECW1,MAD1L1,LASP1,SNX11,TMEM176A,M6PR,KLHL13,CYP26B1,ICA1,DBNDD1,ALS2,CASP10,CFLAR,...,WASIR1.1,IL3RA.1,SHOX.1,ASMT.1,AKAP17A.1,CSF2RA.1,CRLF2.1,ZBED1.1,RNA5SP498.1,TRPC6P.1,RP13-297E16.4.1,DHRSX-IT1.1,CD99P1.1,RPL14P5.1,LINC00685.1,DDX11L16.1,TCEB1P24.1,LL0YNC03-29C1.1.1,KRT18P53.1,LINC00102.1,RP13-297E16.5.1,FABP5P13.1,ASMTL-AS1.1,LINC00106.1,DPH3P2.1,RP11-309M23.1.1,AMDP1.1,BX649553.1.1,BX649553.2.1,BX649553.3.1,BX649553.4.1,MIR3690.1,AJ271736.10.1,Metazoa_SRP.305,AJ271736.1.1,MIR6089.1,RP13-465B17.5.1,RP13-465B17.4.1,project_id_name,project_id
0,1.829525,0.000000,2.062846,0.913543,0.809255,0.365063,1.315347,1.936558,0.657487,1.437908,0.551174,0.719120,1.393872,1.118917,1.987483,0.000000,1.393437,0.257243,1.161696,1.141140,0.0,1.644513,1.765476,2.090856,0.549029,0.718534,0.0,0.0,0.750320,1.912752,1.141035,1.480509,1.567273,0.164270,0.101726,1.313405,1.158674,1.141063,0.798311,1.360485,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TCGA-OV,10
1,1.541309,0.000000,1.815986,0.593689,0.557331,0.608742,1.376983,1.592561,0.569209,1.103958,0.478385,1.033488,1.008026,0.758012,1.681421,1.453353,1.018066,0.223281,0.788680,0.693349,0.0,2.098755,1.643610,2.391891,0.884490,0.000000,0.0,0.0,1.128794,1.836294,0.778508,1.743826,1.415378,0.000000,0.173718,0.749825,1.539502,0.676204,0.761320,0.964733,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TCGA-OV,10
2,1.282789,0.000000,1.895928,0.571531,0.332523,0.576545,1.171834,1.351888,0.836259,1.423673,0.200690,0.874702,0.876875,0.904634,1.109651,0.617534,0.904555,0.000000,0.724887,0.834916,0.0,1.567345,1.350662,2.029576,0.246144,2.193634,0.0,0.0,0.820252,1.581194,0.813868,1.547321,1.086540,0.000000,0.000000,0.464639,1.080810,0.456927,0.652686,0.725820,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TCGA-OV,10
3,1.388260,0.000000,1.905475,0.399374,0.259750,0.219200,0.494936,1.822393,0.619865,0.793788,0.221321,0.567275,1.334915,0.984844,1.915951,0.458862,0.611663,0.000000,0.341496,0.735198,0.0,1.955291,1.635719,2.151499,0.000000,1.956215,0.0,0.0,0.867816,1.938688,1.049283,2.416171,1.523516,0.000000,0.000000,0.663285,0.998184,0.468721,0.456932,0.606861,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TCGA-OV,10
4,1.389160,0.530588,2.316735,0.349455,0.189075,0.623428,0.801534,1.685248,0.784901,0.922292,0.489743,0.968513,1.126255,0.844110,0.942073,0.000000,0.995927,0.169634,0.783794,0.614325,0.0,2.429317,1.840638,2.187733,0.238939,1.196076,0.0,0.0,1.040287,1.824699,0.927491,1.818541,1.317428,0.000000,0.000000,1.098857,1.070776,0.503460,0.218645,0.819567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TCGA-OV,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,1.897300,0.000000,1.829995,0.656801,0.001333,0.528604,2.618544,1.993496,1.487510,1.049062,0.000000,0.258848,1.455348,0.853410,1.215961,0.000000,0.571925,0.644190,0.536399,0.215012,0.0,1.704930,2.033992,2.059344,0.000000,0.000000,0.0,0.0,0.811806,1.861269,1.084387,2.724172,1.159409,0.000000,0.000000,0.193960,1.282674,0.685311,0.463778,0.899169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [30]:
def gene_cancer(cancer_name,top_gene_num=10):
  cancer = TC1data15[TC1data15['project_id_name'] == cancer_name].copy()
  cancer['project_id'] = cancer['project_id'].apply(lambda x: 1)
  non_cancer = TC1data15[TC1data15['project_id_name'] != cancer_name].copy().sample(n = 300) 
  non_cancer['project_id'] = non_cancer['project_id'].apply(lambda x: 0)
  data_cancer = cancer.append(non_cancer)
  # define the model
  model = RandomForestClassifier(n_estimators=100, n_jobs=-1)
  # fit the model
  model.fit(data_cancer.iloc[:,:-2], data_cancer['project_id'])
  importance = model.feature_importances_
  feature = pd.DataFrame()
  feature['name'] = TC1data15.columns[:-2]
  feature['importance'] = importance
  feature_sort = feature.sort_values(by='importance',ascending=False)
  feature_sort_selected = feature_sort.iloc[:top_gene_num,:]
  return feature_sort_selected

In [39]:
np.random.seed(123)
cancer_name = 'TCGA-PRAD'
cancer = TC1data15[TC1data15['project_id_name'] == cancer_name].copy()
cancer['project_id'] = cancer['project_id'].apply(lambda x: 1)
non_cancer = TC1data15[TC1data15['project_id_name'] != cancer_name].copy().sample(n = 300) 
non_cancer['project_id'] = non_cancer['project_id'].apply(lambda x: 0)
data_cancer = cancer.append(non_cancer)
model = RandomForestClassifier(n_estimators=100, n_jobs=-1)
  # fit the model
model.fit(data_cancer.iloc[:,:-2], data_cancer['project_id'])
y_pred = model.predict(data_cancer.iloc[:,:-2])
np.mean(y_pred == data_cancer['project_id'])

1.0

# TCGA-PRAD

In [52]:
np.random.seed(123)
gene_cancer('TCGA-PRAD',10)

,name,importance
21705,TRGC1,0.021603
17106,CPNE4,0.020607
10081,STEAP2,0.020351
44881,AP000438.2,0.018979
44755,ZNF350,0.018920
2780,NME4,0.015973
5625,ABCC4,0.012116
27802,LINC01509,0.011825
43339,FLJ20021,0.011425
8243,KLK3,0.011199


# TCGA-THCA

In [53]:
np.random.seed(123)
gene_cancer('TCGA-THCA',10)

,name,importance
47770,RP11-274H2.5,0.020242
34234,PTCSC2,0.019735
14160,TCERG1L,0.016931
16808,SLC4A5,0.016020
15178,IDH2,0.015731
8954,IGSF1,0.015437
4444,ST3GAL5,0.014778
6841,CNTN6,0.014727
8871,CDCA5,0.013716
1900,KIF4A,0.012915


# TCGA-OV

In [54]:
np.random.seed(123)
gene_cancer('TCGA-OV',10)

,name,importance
57644,FP671120.6,0.021880
51623,RP11-380M21.2,0.015537
3232,NHP2P1,0.015264
58500,RP11-2K6.1,0.014679
35386,BANF1P3,0.014610
50900,MIR4478,0.014173
5294,RAMP3,0.013678
30612,RPS18P12,0.013114
32979,RP11-761N21.2,0.010731
32783,RPL5P34,0.010717


# TCGA-LUAD

In [55]:
np.random.seed(123)
gene_cancer('TCGA-LUAD',10)

,name,importance
17076,SFTA2,0.017978
5428,C4BPA,0.015095
15981,SFTPA2,0.014643
9208,GGTLC1,0.012997
34381,RP11-59D5__B.2,0.010289
12465,SFTPB,0.008252
7269,NKX2-1,0.007987
10583,SCGB3A1,0.007936
10198,ESYT3,0.007848
6445,NAPSA,0.007670


# TCGA-COAD

In [56]:
np.random.seed(123)
gene_cancer('TCGA-COAD',10)

,name,importance
3935,APOBEC1,0.015949
9748,GAL3ST2,0.012537
12036,LINC00483,0.012353
1893,CERS4,0.012286
20012,TRIM31,0.011761
1225,LLGL2,0.011101
51764,AC024592.9,0.010795
8877,MDH2,0.010229
11633,CDX2,0.009769
13263,ATOH1,0.009724


# TCGA-LIHC

In [57]:
np.random.seed(123)
gene_cancer('TCGA-LIHC',10)

,name,importance
1634,APOB,0.019583
1799,F11,0.018195
2136,SERPIND1,0.017971
23156,APOC1P1,0.017269
95,PON1,0.017042
252,CD9,0.012147
8355,F13B,0.011406
9440,ITIH2,0.010944
6648,APCS,0.010540
11339,KLKB1,0.010470


# TCGA-BRCA

In [58]:
np.random.seed(123)
gene_cancer('TCGA-BRCA',10)

,name,importance
2925,TRPS1,0.022681
49229,RP11-401O9.3,0.010730
3665,TBC1D9,0.010065
25694,RP11-428L9.2,0.009479
7391,LMX1B,0.009254
5771,ID1,0.008982
4013,SAYSD1,0.008950
26416,RP11-123K19.1,0.008021
46769,CTD-3032H12.2,0.007840
17689,MB,0.007604


# TCGA-STAD

In [59]:
np.random.seed(123)
gene_cancer('TCGA-STAD',10)

,name,importance
54958,RP11-174N3.4,0.018834
37411,CTD-2184D3.1,0.016162
5790,UROD,0.014576
48632,RP11-750B16.1,0.014133
23916,RP3-340B19.2,0.013708
28168,AC139452.2,0.013535
14841,RP1-278E11.3,0.013276
52515,RP11-3P17.5,0.013177
57365,FP236383.12,0.011070
25348,RNA5SP498,0.010114


# TCGA-UCEC

In [60]:
np.random.seed(123)
gene_cancer('TCGA-UCEC',10)

,name,importance
9987,FGF18,0.014015
10932,MSX1,0.010787
4865,ELL2,0.010397
12020,PBX3,0.010102
54907,LL21NC02-21A1.1,0.008687
38508,CEBPA,0.008425
12373,RAB31,0.008248
6407,ZNF428,0.007958
5537,PACSIN1,0.007518
10636,FAM171A2,0.007357


# TCGA-KIRC

In [61]:
np.random.seed(123)
gene_cancer('TCGA-KIRC',10)

,name,importance
28518,RP11-807H17.1,0.018217
29214,LINC01428,0.017579
7744,ENPEP,0.017433
22905,LINC00887,0.015961
20842,RNU6-339P,0.014603
15187,TMEM30B,0.013393
14753,CDH5,0.012996
10564,RECQL4,0.012120
44359,RP11-283I3.4,0.010903
17504,HOXA4,0.010769


# TCGA-LGG

In [62]:
np.random.seed(123)
gene_cancer('TCGA-LGG',10)

,name,importance
2875,SCG3,0.019736
3139,CACNG7,0.019735
10876,RFTN2,0.019669
15186,FAM181B,0.019540
28317,AC018647.3,0.011427
7879,SLAIN1,0.011377
2121,HCN2,0.011214
5475,GDAP1L1,0.010975
17885,LRRTM3,0.010698
6124,ATP1B2,0.010583
